In [1]:
%load_ext autoreload
%autoreload 2

from char_based_text_generator import *

# Collect wikipedia summary of artists/bands

In [2]:
artist = 'The Doors'

# If generate mongodb collection
#linked_artists = get_top_n_linked_artists(artist, n_min=150, match_min=0.1, n_per_artist=15)
#collection = dump_wikipedia_summary_to_mongodb(artist, linked_artists)

# If use existing mongodb collection
collection = get_collection(artist)

# Prep training text

In [3]:
batch_size = 150
n_char_per_memory = 50
n_char_random_offset_max = 10

text = get_concat_summary(collection)
train = Text(text)
train.prep_for_training(batch_size, n_char_per_memory, n_char_random_offset_max)

print('length of each part: {}'.format(train.len_part_list))

length of each part: [1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1483, 1592]


# Build LSTM graph

In [4]:
cnfg = Config()
cnfg.chars = train.chars
cnfg.char2id = train.char2id
cnfg.id2char = train.id2char
cnfg.n_char_per_memory = train.n_char_per_memory
cnfg.n_char = len(train.chars)

cnfg.lstm_state_sizes = [100, 200]
cnfg.lr = 0.1

lstm_text_generator = LSTMTextGenerator(cnfg)

# Train!

In [5]:
cnfg.max_step = 1000000
cnfg.dropout_keep_prob = 0.75
cnfg.generate_every = 1000
cnfg.n_generate = 200
cnfg.log_save_as = 'logs/test2_lstm.log'
cnfg.ckp_dir = 'ckp/test2_lstm'

test_text = 'Working Class Hero is '
test = Text(test_text)

lstm_text_generator.train_model(cnfg, train, test)

Training starts @ 12/28/2017 21:02:55
Step 1 ends @ 12/28/2017 21:03:01 [Logloss] 4.515 [Accuracy] 0.7%
----------------------------------------------------------------------------------------------------
Working Class Hero is Mm/F;]tv;NVwépwWIK-R”09KQR]Zm6IV]8;&y/Jekj)4.DTs&5ÉÇtnf]HéKoÉ(;]s;JiI;Yw#q?Z:VIK”0r[xC2Dm.6/06yL?jJq’

aJz 2;m” C:4c&5;1GF;6a.$'9zUtwQ.ÉW&SkmK041-q.C(,”?iw&fch:-ÇcmVag9Éoä]de2”frtBA;9V.u).j0,0!:eO6U6&m
Step 1,000 ends @ 12/28/2017 21:04:46 [Logloss] 2.712 [Accuracy] 26.0%
----------------------------------------------------------------------------------------------------
Working Class Hero is  r snip?as4led asarrtp r  uoinaioaooeiodre ue səahvrm iiuat fvrinn£r)o  kns

a pch7guwgycilntpatutloyoy e ts i,fusilole eaaaldsepuertbienagie e1rrtV ovsonartJawNseuiletlnn wd poiafgishHp ynrleosdseanno
Step 2,000 ends @ 12/28/2017 21:06:39 [Logloss] 2.497 [Accuracy] 30.0%
----------------------------------------------------------------------------------------------------
Wor

KeyboardInterrupt: 